In [1]:
import matplotlib.pyplot as plt
from joblib import load, dump
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import numpy as np
import warnings, os


import seaborn as sns
import scipy.stats as stats
import tensorflow as tf

import aggmap
from aggmap import loadmap, aggmodel


import os
os.environ["CUDA_VISIBLE_DEVICES"]="5"

## fix random seed to get repeatale results
seed = 666
np.random.seed(seed)
tf.compat.v1.set_random_seed(seed)
warnings.filterwarnings("ignore")


In [2]:
results_save_folder = './model_results/'
if not os.path.exists(results_save_folder):
    os.makedirs(results_save_folder)

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data() #(x_train, y_train), (x_test, y_test)
mnist_labels_dict = {0: 'T-shirt/top',1: 'Trouser',2: 'Pullover',
                     3: 'Dress',4: 'Coat', 5: 'Sandal', 
                     6: 'Shirt', 7: 'Sneaker', 8: 'Bag', 9: 'Ankle boot'}

In [4]:
Y = pd.get_dummies(y_train).values.astype(float)
testY = pd.get_dummies(y_test).values.astype(float)

In [5]:
data_save_folder = '/raid/shenwanxiang/aggmap/fmnist/correlation/'


#orignal images
X_OR1 = load(os.path.join(data_save_folder, '01_X_OR1.data'))

#random shuffle
X_ORS1 = load(os.path.join(data_save_folder, '02_X_ORS1.data'))

#agg
X_ORSAgg1 = load(os.path.join(data_save_folder, '03_X_ORSAgg1.data'))

#cluster split channel
X_ORSAggC3 = load(os.path.join(data_save_folder, '03_X_ORSAggC3.data'))
X_ORSAggC5 = load(os.path.join(data_save_folder, '04_X_ORSAggC5.data'))
X_ORSAggC10 = load(os.path.join(data_save_folder, '05_X_ORSAggC10.data'))


In [6]:
fmp_types = [ X_OR1,  X_ORS1,  X_ORSAgg1, X_ORSAggC3,  X_ORSAggC5,  X_ORSAggC10]
fmp_names = ['OR1',  'ORS1',  'ORSAgg1',  'ORSAggC3', 'ORSAggC5',  'ORSAggC10']

# fmp_types = [  X_ORSAggC3,  X_ORSAgg1]
# fmp_names = ['ORSAggC3', 'ORSAgg1']

In [ ]:
for name, x in zip(fmp_names, fmp_types) :
    
    X, testX = x[0]/255., x[1]/255.

    trainX, validX, trainY, validY = train_test_split(X, Y, test_size = 0.1, random_state = seed)

    epochs = 100
    patience = 1000
    dense_layers = [256, 128, 64] #
    batch_size = 64
    lr = 1e-4

    dense_avf = 'relu'
    last_avf = 'softmax' #sigmoid in loss

    model = aggmodel.net.AggMapNet(trainX.shape[1:],
                                   n_outputs=trainY.shape[-1], 
                                   conv1_kernel_size = 3,
                                   dense_layers=dense_layers, 
                                   dense_avf = dense_avf, 
                                   last_avf=last_avf)

    opt = tf.keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0) #

    performance = aggmodel.cbks.CLA_EarlyStoppingAndPerformance((trainX, trainY), 
                                                                (validX, validY), 
                                                                patience = patience, 
                                                                criteria = 'val_auc',
                                                                metric = 'ACC',
                                                                last_avf= last_avf,
                                                                verbose = 0)

    model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

    history = model.fit(trainX, trainY, batch_size=batch_size,  # verbose= 2, 
                          epochs=epochs, shuffle = True, 
                          validation_data = (validX, validY), callbacks= [performance]) 

    train_loss, train_acc = model.evaluate(trainX, trainY, verbose=0)

    valid_loss, valid_acc = model.evaluate(validX, validY, verbose=0)

    test_loss, test_acc = model.evaluate(testX, testY, verbose=0)


    res = {'train_size':len(trainX),
           'valid_size': len(validX),
           'test_size': len(testX),
           'dense_layers':dense_layers,
           'dense_avf':dense_avf,
           'last_avf':last_avf, 
           'trainable_params':model.count_params(),
           'batch_size':batch_size,
           'lr':lr,
           'best_epochs_in_valid': performance.best_epoch + 1,
           
           'train_acc':train_acc,
           'valid_best_acc': valid_acc,
           'test_acc': test_acc,
           
           'train_loss':train_loss,
           'valid_best_loss': valid_loss,
           'test_loss': test_loss,}

    pd.DataFrame(history.history).to_csv(os.path.join(results_save_folder, 'training_%s.csv' % name))
    
    pd.DataFrame([res]).to_csv(os.path.join(results_save_folder, 'performance_%s.csv' % name))
    
    model.save(os.path.join(results_save_folder, 'model_%s.h5' % name))
    
    test_report = pd.DataFrame(classification_report(validY, np.round(model.predict(validX)),output_dict = True )).T
    test_report.to_csv(os.path.join(results_save_folder, 'test_report_%s.csv' % name))

Train on 54000 samples, validate on 6000 samples
Epoch 1/100
54000/54000 [==============================] - 12s 223us/sample - loss: 0.8228 - accuracy: 0.7099 - val_loss: 0.5101 - val_accuracy: 0.8100
Epoch 2/100
54000/54000 [==============================] - 8s 147us/sample - loss: 0.4642 - accuracy: 0.8311 - val_loss: 0.4251 - val_accuracy: 0.8437
Epoch 3/100
54000/54000 [==============================] - 8s 149us/sample - loss: 0.3951 - accuracy: 0.8578 - val_loss: 0.3703 - val_accuracy: 0.8658
Epoch 4/100
54000/54000 [==============================] - 8s 152us/sample - loss: 0.3576 - accuracy: 0.8710 - val_loss: 0.3647 - val_accuracy: 0.8643
Epoch 5/100
54000/54000 [==============================] - 8s 150us/sample - loss: 0.3287 - accuracy: 0.8815 - val_loss: 0.3336 - val_accuracy: 0.8787
Epoch 6/100
54000/54000 [==============================] - 8s 153us/sample - loss: 0.3087 - accuracy: 0.8886 - val_loss: 0.3048 - val_accuracy: 0.8907
Epoch 7/100
54000/54000 [===================